In [1]:
import numpy as np
import matplotlib.pyplot as plt

import matplotlib.style
import matplotlib as mpl
mpl.style.use('classic')

import baraffe_tables
from baraffe_tables.table_search import baraffe_table_search

from astropy.table import Table
from astropy import units as u
from astropy.constants import G

import mesa_helper as mh
import os
import shutil

%matplotlib inline

In [2]:
mJtomSun = u.jupiterMass.to(u.solMass)
mJtoGrams = u.jupiterMass.to(u.g)

rJtorSun = u.jupiterRad.to(u.solRad)
rJtoCm = u.jupiterRad.to(u.cm)

print(mJtomSun)
print(mJtoGrams)
print(rJtorSun)
print(rJtoCm)


0.0009545942339693249
1.8981245973360504e+30
0.10276268506540176
7149200000.0


In [3]:
#jupiter masses
Minit = np.array((20.0, 14.0, 9.8, 6.9, 4.8, 3.4, 2.3, 1.6, 1.1, 0.82, 0.57, 0.4, 0.28, 0.2))

#jupiter radii
Rinit = np.array((5.0, 2.0, 1.0, 0.5, 0.2, 0.1, 0.05))

In [4]:
# make directories for all planet initial masses
for i, m in enumerate(Minit[4:]):
    os.system("cp -r template_makeplanet Minit={0}Mj".format(m))

#edit inlist_create, inlist_evolve, rn
for i, m in enumerate(Minit[4:]):
    print(m)
    #edit inlists
    create_model_filename = "planet_create_{0}_Mj_{1}_Rj.mod".format(m, Rinit[0])
    evolve_model_filename = "planet_evolve_{0}_Mj_{1}_Rj.mod".format(m, Rinit[0])

    inlist_create = mh.Inlist('./Minit={0}Mj/inlist_create'.format(m))
    inlist_evolve = mh.Inlist('./Minit={0}Mj/inlist_evolve'.format(m))

    #edit inlist_create options
    inlist_create.set_option('mass_in_gm_for_create_initial_model', m*mJtoGrams)
    # start with initial radius equal to 5 Rj. This will need to be revised downward for less massive planets.
    # If it needs to be revised to be very small, may also need to change kap_lowT_prefix option.
    inlist_create.set_option('radius_in_cm_for_create_initial_model', Rinit[0]*rJtoCm)
    inlist_create.set_option('save_model_filename', create_model_filename)

    #edit inlist_evolve options
    inlist_evolve.set_option('load_model_filename', create_model_filename)
    inlist_evolve.set_option('save_model_filename', evolve_model_filename)
    
    # edit rn script
    f = open('./Minit={0}Mj/rn'.format(m), 'r')
    g = f.read()
    f.close()
    #print(g)
    g = g.replace("<<create_model_filename>>", create_model_filename)
    g = g.replace("<<evolve_model_filename>>", evolve_model_filename)
    
    h = open('./Minit={0}Mj/rn_temp'.format(m), 'w')
    h.write(g)
    h.close()
    shutil.copyfile('./Minit={0}Mj/rn_temp'.format(m),'./Minit={0}Mj/rn'.format(m))
    
    # execute mk script
    os.chdir('./Minit={0}Mj/'.format(m))
    os.system('./mk'.format(m))
    os.chdir('..')


4.8
Set mass_in_gm_for_create_initial_model to 9.111d30
Set radius_in_cm_for_create_initial_model to 3.5746d10
Set save_model_filename to 'planet_create_4.8_Mj_5.0_Rj.mod'
Set load_model_filename to 'planet_create_4.8_Mj_5.0_Rj.mod'
Set save_model_filename to 'planet_evolve_4.8_Mj_5.0_Rj.mod'
gfortran -fopenmp -o ../star  run_star_extras.o run_star.o  run.o  -L/Users/Emily/mesa-r24.03.1/lib -lstar -lgyre -lionization -latm -lcolors -lturb -lstar_data -lnet -leos -lkap -lrates -lneu -lchem -linterp_2d -linterp_1d -lnum -lauto_diff -lhdf5io -lmtx -lconst -lmath -lutils `mesasdk_crmath_link` `mesasdk_lapack95_link` `mesasdk_lapack_link` `mesasdk_blas_link` `mesasdk_hdf5_link`  `mesasdk_pgplot_link` -lz 
3.4
Set mass_in_gm_for_create_initial_model to 6.45362d30
Set radius_in_cm_for_create_initial_model to 3.5746d10
Set save_model_filename to 'planet_create_3.4_Mj_5.0_Rj.mod'
Set load_model_filename to 'planet_create_3.4_Mj_5.0_Rj.mod'
Set save_model_filename to 'planet_evolve_3.4_Mj_5.0_Rj

In [5]:
def update_radius(m, rold, rnew):
    """
    m in Mj, rnew in Rj
    Assumes that this will be run from within the relevant model folder
    """
    
    #edit inlists
    old_create_model_filename = "planet_create_{0}_Mj_{1}_Rj.mod".format(m, rold)
    old_evolve_model_filename = "planet_evolve_{0}_Mj_{1}_Rj.mod".format(m, rold)
    
    create_model_filename = "planet_create_{0}_Mj_{1}_Rj.mod".format(m, rnew)
    evolve_model_filename = "planet_evolve_{0}_Mj_{1}_Rj.mod".format(m, rnew)

    inlist_create = mh.Inlist('./inlist_create')
    inlist_evolve = mh.Inlist('./inlist_evolve')

    #edit inlist_create options
    inlist_create.set_option('mass_in_gm_for_create_initial_model', m*mJtoGrams)
    inlist_create.set_option('radius_in_cm_for_create_initial_model', rnew*rJtoCm)
    inlist_create.set_option('save_model_filename', create_model_filename)

    #edit inlist_evolve options
    inlist_evolve.set_option('load_model_filename', create_model_filename)
    inlist_evolve.set_option('save_model_filename', evolve_model_filename)
    
    # edit rn script
    f = open('./rn', 'r')
    g = f.read()
    f.close()

    g = g.replace(old_create_model_filename, create_model_filename)
    g = g.replace(old_evolve_model_filename, evolve_model_filename)
    
    h = open('./rn_temp', 'w')
    h.write(g)
    h.close()
    shutil.copyfile('./rn_temp','./rn')
    
    return


In [6]:
# need to try stepping through R values until code runs
for i, m in enumerate(Minit[4:]):
    print("")
    print("m is {0} Mj".format(m))
    print("")
    os.chdir('./Minit={0}Mj/'.format(m))
    os.system('pwd')

    for j, r in enumerate(Rinit[:-1]):
    
        print("r is {0} Rj".format(r))

        create_model_filename = "planet_create_{0}_Mj_{1}_Rj.mod".format(m, r)
        evolve_model_filename = "planet_evolve_{0}_Mj_{1}_Rj.mod".format(m, r)

        # try running planet at this radius
       
        os.system('./rn')

        #test if "create" and "evolve" models exist, indicating success
        test_create = os.path.isfile(create_model_filename)
        test_evolve = os.path.isfile(evolve_model_filename)

        print("test_create is {0} and test_evolve is {1}".format(test_create, test_evolve))

        if test_create is False:
            print("model was not created at mass {0} Mj, radius {1} Rj".format(m, r))
            os.system('rm ./LOGS/*')
            os.system('rm ./photos/*')

            inlist_create = mh.Inlist('./inlist_create')
            inlist_create.set_option('kap_lowT_prefix', 'lowT_fa05_gs98')
        
            os.system('./rn')
            test_create = os.path.isfile(create_model_filename)
            test_evolve = os.path.isfile(evolve_model_filename)

            print("changed low T opacities and now test_create is {0} and test_evolve is {1} at m = {2} Mj, r = {3} Rj".format(test_create, test_evolve, m, r))

        elif test_create is True and test_evolve is False:
            print("model did not evolve.")
            try:
                history = Table.read("./LOGS/history.data", format="ascii", header_start=4, data_start=5)
                max_age_reached = history['star_age'][-1]
                print("max age reached is {0} yr".format(max_age_reached))
            except FileNotFoundError:
                print("no history file")

            os.system('rm ./LOGS/*')
            os.system('rm ./photos/*')
            update_radius(m, r, Rinit[j+1])
            print("trying at new radius {0} Rj".format(Rinit[j+1]))
            os.system('./rn')
            test_create = os.path.isfile(create_model_filename)
            test_evolve = os.path.isfile(evolve_model_filename)
            
        elif test_create is True and test_evolve is True:
            #success, no need to try further r values
            break
    
    os.chdir('..')
    


m is 4.8 Mj

/Users/Emily/Documents/astro/giant_planets/MESAtests/reproducePaxton+2011fig18/Minit=4.8Mj
r is 5.0 Rj
DATE: 2024-07-25
TIME: 13:48:24
run inlist_create_header
DATE: 2024-07-25
TIME: 13:48:24
 reading user weak rate file /Users/Emily/mesa-r24.03.1/data/rates_data/rate_tables/S13_r_be7_wk_li7.h5
 version_number r24.03.1
 read inlist_create
 create initial model
                         create initial model iteration           1
      log10(L/Lsun)=  -3.07996188E+00
               Mass=   1.63670084E+30
             Radius=   6.39715627E+10
 
      log10(L/Lsun)=  -2.97569028E+00
               Mass=   1.77441956E+30
             Radius=   6.66204450E+10
 
      log10(L/Lsun)=  -2.25626134E+00
               Mass=   1.63841424E+30
             Radius=   6.61941755E+10
 
                         create initial model iteration           2
      log10(L/Lsun)=  -2.57591491E+00
               Mass=   2.68420595E+30
             Radius=   8.41372015E+10
 
      log10(L/Lsun)=  -

rm: ./LOGS/*: No such file or directory
rm: ./photos/*: No such file or directory


changed low T opacities and now test_create is False and test_evolve is False at m = 4.8 Mj, r = 2.0 Rj
r is 1.0 Rj
DATE: 2024-07-25
TIME: 13:56:42
run inlist_create_header
DATE: 2024-07-25
TIME: 13:56:42
 reading user weak rate file /Users/Emily/mesa-r24.03.1/data/rates_data/rate_tables/S13_r_be7_wk_li7.h5
 version_number r24.03.1
 read inlist_create
 create initial model
                         create initial model iteration           1
 failed in kap_get get_kap_from_rhoT
 failed in kap_get get_kap_from_rhoT
 failed in kap_get get_kap_from_rhoT
 failed in kap_get get_kap_from_rhoT
 failed in kap_get get_kap_from_rhoT
 failed in kap_get get_kap_from_rhoT
 failed in kap_get get_kap_from_rhoT
 failed in kap_get get_kap_from_rhoT
 failed in kap_get get_kap_from_rhoT
 failed in kap_get get_kap_from_rhoT
 failed in kap_get get_kap_from_rhoT
 failed in kap_get get_kap_from_rhoT
 failed in kap_get get_kap_from_rhoT
 failed in kap_get get_kap_from_rhoT
 failed in kap_get get_kap_from_rhoT
 

rm: ./LOGS/*: No such file or directory
rm: ./photos/*: No such file or directory


 reading user weak rate file /Users/Emily/mesa-r24.03.1/data/rates_data/rate_tables/S13_r_be7_wk_li7.h5
 version_number r24.03.1
 read inlist_create
 create initial model
                         create initial model iteration           1
 failed in kap_get get_kap_from_rhoT
 failed in kap_get get_kap_from_rhoT
 failed in kap_get get_kap_from_rhoT
 failed in kap_get get_kap_from_rhoT
 failed in kap_get get_kap_from_rhoT
 failed in kap_get get_kap_from_rhoT
 failed in kap_get get_kap_from_rhoT
 failed in kap_get get_kap_from_rhoT
 failed in kap_get get_kap_from_rhoT
 failed in kap_get get_kap_from_rhoT
 failed in kap_get get_kap_from_rhoT
 failed in kap_get get_kap_from_rhoT
 failed in kap_get get_kap_from_rhoT
 failed in kap_get get_kap_from_rhoT
 failed in kap_get get_kap_from_rhoT
 temp too low in integration
DATE: 2024-07-25
TIME: 13:56:55

******************************************************************
failed to create planet_create_4.8_Mj_2.0_Rj.mod when running inlist_create_h

shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
sh: ./rn: No such file or directory
shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
rm: ./LOGS/*: No such file or directory
shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
rm: ./photos/*: No such file or directory


FileNotFoundError: [Errno 2] No such file or directory: './inlist_create'